# using dask to handle a lot of data


In [43]:
import dask
from dask.diagnostics import ProgressBar
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster, progress
import us
import os
import numpy as np

In [58]:
# total_mb = 0


In [63]:
total = 0
for file in os.listdir("../data/citibike-tripdata-nyc/"): 
    if file.endswith(".csv"):
        file_stats = os.stat(f"../data/citibike-tripdata-nyc/{file}")
        total_mb = round(file_stats.st_size)
#         print(f'{file} is {total_mb} mb')
        total +=total_mb
print(str(total/1e9) + " in gb")

22.452526791 in gb


In [64]:
try:
    cluster
except NameError:
    # create LocalCluster w/ dask
    cluster = LocalCluster()
    client = Client(cluster)
    # then look at diagnostic dash here http://localhost:8787/status or http://127.0.0.1:8787/status
    client
    # # to turn off
    # client.shutdown()

In [65]:
client

Client Scheduler: tcp://127.0.0.1:52383 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 16 Memory: 68.72 GB


In [66]:
# combine files
@dask.delayed
def read_files(csv_dir):
    combined_df = []
    for item in os.listdir(csv_dir):
        if item.endswith(".csv"):
            file_name = csv_dir + item
            print(file_name)
            df = dd.read_csv(file_name, assume_missing=True, dtype={'end_station_id': 'object', 'start_station_id': 'object'})
            combined_df.append(df)
    combined_df = dd.concat(combined_df)
    return combined_df

In [67]:
future_nyc_citibike = client.compute(read_files("../data/citibike-tripdata-nyc/"))
progress(future_nyc_citibike)

VBox()

In [68]:
future_nyc_citibike

<Future: finished, type: dask.DataFrame, key: read_files-fc33a173-c6b1-4093-9aac-f5e55561d4a1>

In [69]:
nyc_df = future_nyc_citibike.result()
nyc_df.head(2)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,...,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
0,362.0,2017-09-01 00:00:17,2017-09-01 00:06:19,3331.0,Riverside Dr & W 104 St,40.801343,-73.971146,3328.0,W 100 St & Manhattan Ave,40.795000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,188.0,2017-09-01 00:00:21,2017-09-01 00:03:30,3101.0,N 12 St & Bedford Ave,40.720798,-73.954847,3100.0,Nassau Ave & Newell St,40.724813,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
future_jc_citibike = client.compute(read_files("../data/citibike-tripdata-jc/"))
progress(future_jc_citibike)

VBox()

In [27]:
%%time 
# checkout missing data situation 
# len(nyc_df['starttime'])
len(nyc_df.index)

CPU times: user 6.54 s, sys: 747 ms, total: 7.29 s
Wall time: 1min 33s


119519535

In [70]:
%%time

# total trips per month
# most popular start station
# most popular routes (start + end station)
# average trip length by month? by day of week?

nyc_df.groupby(['starttime']).count().compute()


distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker


KilledWorker: ("('dataframe-groupby-count-combine-136a8dfca7804d3cf11d2f2bbca131ae', 2, 4, 0)", <Worker 'tcp://127.0.0.1:53039', name: 0, memory: 0, processing: 9>)

In [ ]:
# states = [i.abbr for i in us.STATES]
# nyc_residents = dd.read_csv("../data/covid_19_data.csv")
# final_df = []


# for state in states:
#     #     maybe you need path as well?
#     path = os.getcwd()
#     filename = str(path+"/"+state+".csv")
#     current_state_df = dd.read_csv(f"{filename}", header=[])
#     merge_df = df.merge(left=nyc_residents, right=current_state_df, how="inner", on=['pid', 'addrid'])
#     final_df.append(merge_df)
# final_df = pd.concat(final_df)


In [ ]:
# "https://stackoverflow.com/questions/65619777/display-progress-on-dask-computesomething-call"

tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <tornado.platform.asyncio.AsyncIOMainLoop object at 0x105be53c8>>, <Task finished coro=<MultiProgressBar.listen() done, defined at /Users/spak/.pyenv/versions/3.6.5/envs/hsiling/lib/python3.6/site-packages/distributed/diagnostics/progressbar.py:238> exception=OSError('Timed out trying to connect to tcp://127.0.0.1:52383 after 10 s',)>)
Traceback (most recent call last):
  File "/Users/spak/.pyenv/versions/3.6.5/envs/hsiling/lib/python3.6/site-packages/distributed/comm/core.py", line 288, in connect
    timeout=min(intermediate_cap, time_left()),
  File "/Users/spak/.pyenv/versions/3.6.5/lib/python3.6/asyncio/tasks.py", line 362, in wait_for
    raise futures.TimeoutError()
concurrent.futures._base.TimeoutError

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/spak/.pyenv/versions/3.6.5/envs/hsiling/li